In [42]:
import pandas as pd
import numpy as np
import os

In [43]:
# creating the dataframes
schoolsdf = pd.read_csv(os.path.join("raw_data","schools_complete.csv"))
studentsdf =  pd.read_csv(os.path.join("raw_data","students_complete.csv"))
schoolsdf.describe()


,School ID,size,budget
count,15.000000,15.000000,1.500000e+01
mean,7.000000,2611.333333,1.643295e+06
std,4.472136,1420.915282,9.347763e+05
min,0.000000,427.000000,2.480870e+05
25%,3.500000,1698.000000,1.046265e+06
50%,7.000000,2283.000000,1.319574e+06
75%,10.500000,3474.000000,2.228999e+06
max,14.000000,4976.000000,3.124928e+06


In [44]:
# This cell is for learning about the Schools Data
#schoolsdf.describe()
schoolsdf.columns

Index(['School ID', 'name', 'type', 'size', 'budget'], dtype='object')

In [45]:
#This cell is for learningabout the Students Data
#studentsdf.describe()
studentsdf.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [46]:
studentsdf.describe()


,Student ID,reading_score,math_score
count,39170.000000,39170.00000,39170.000000
mean,19584.500000,81.87784,78.985371
std,11307.549359,10.23958,12.309968
min,0.000000,63.00000,55.000000
25%,9792.250000,73.00000,69.000000
50%,19584.500000,82.00000,79.000000
75%,29376.750000,91.00000,89.000000
max,39169.000000,99.00000,99.000000


In [67]:
TotalSchools = schoolsdf['School ID'].count()
TotalStudents = studentsdf['Student ID'].count()
Totalbudget = schoolsdf['budget'].sum()
AvgMathScore = studentsdf['math_score'].mean()
AvgReadingScore = studentsdf['reading_score'].mean()
PerMathPass= (len(studentsdf.loc[studentsdf['math_score'] >65])/TotalStudents)*100
PerReadingPass = (len(studentsdf.loc[studentsdf['reading_score'] >65])/TotalStudents)*100


DistrictSummary= {'Total Schools': TotalSchools,'Total Students' : TotalStudents,'Total budget': Totalbudget,
                                  'Average Math Score': AvgMathScore, 'Average Reading Score':AvgReadingScore,
                                 '% Passing Math': PerMathPass, '% Passing Reading' :PerReadingPass}
#pd.DataFrame(d.items(), columns=['id', 'cost','name'])
DistrictSummaryDf = pd.DataFrame(DistrictSummary.items(), columns =DistrictSummary.keys())
DistrictSummaryDf

ValueError: DataFrame constructor not properly called!